# WIBARAB Data Validation workflow

**Author:** Daniel Schopper    
**Description:** This notebook automates the validation of WIBARAB FeatureDB documents. When finished, it should be ported to a non-interactive script which is triggered by a github action.    
**Last Change:** 2023-01-25

## Step 0: Setup

In [7]:
import io
import os
import requests
import pathlib

from pathlib import Path
from urllib.parse import urlsplit
from saxonpy import PySaxonProcessor, PyXdmValue
from zipfile import ZipFile

from datetime import datetime
# from inspect import getmembers, signature

In [8]:
def transform(s, xsl, o, parameters=[]):
    # no change in behavior
    proc.set_configuration_property("xi", "on")
    saxon = proc.new_xslt_processor()
    saxon.set_source(file_name=os.path.abspath(s))
    for i in parameters:
        saxon.set_parameter(name=i, value=proc.make_string_value(parameters[i]))
    saxon.compile_stylesheet(stylesheet_file=os.path.abspath(xsl))
    saxon.set_output_file(os.path.abspath(o))
    saxon.transform_to_string()
    if saxon.exception_occurred():
        #print(getmembers(saxon))
        saxon.get_error_message(0)
        for i in range(saxon.exception_count()-1):
            print(saxon.get_error_message(0))
        print(os.path.abspath(s)+" - "+os.path.abspath(xsl)+" -> "+os.path.abspath(o)+" failed")
    return o # for chaining


In [2]:
## Step 1: Expand References

The WIBARAB Feature Documents contain 